In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir chap6
%cd ./chap6

mkdir: cannot create directory ‘chap6’: File exists
/content/chap6


In [ ]:
# インストール
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

In [ ]:
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# Livedoorのニュース記事分類
BERTのファインチューニングと性能評価  
手順：
1.   データダウンロード
2.     データ前処理  
2-1.  データローダ作成  
2-2.  データ前処理  
2-3.  データセットの分割  
3.   PyTorch Lightningによるファインチューニングとテスト  
4.   モデルの評価


## 1. データのダウンロード
ポイント：livedoorのニュースコーパスはファイル数が数千ある。Colabからgoogle driveへのアクセスは、あまり早くないため、Colabのシステムのディレクトリに配置する。

In [ ]:
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz 
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz 


--2021-08-06 07:34:35--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

ldcc-20140209.tar.g 100%[===================>]   8.44M  17.8MB/s    in 0.5s    

2021-08-06 07:34:36 (17.8 MB/s) - ‘ldcc-20140209.tar.gz.1’ saved [8855190/8855190]



In [ ]:
# 中身のファイルデータの確認
!cat ./text/it-life-hack/it-life-hack-6342280.txt

http://news.livedoor.com/article/detail/6342280/
2012-03-06T13:00:00+0900
USB3.0対応で爆速データ転送！　9倍速のリーダー／ライター登場
USB3.0が登場してから今年で4年目となるがパソコン側でのUSB3.0ポート搭載が進んで来ても対応機器がなかなか充実していない現状がある。そんな中で新しく高速な読み取りが可能なメモリーカードリーダー／ライターが登場した。

バッファローコクヨサプライがUSB3.0対応のカードリーダー／ライターを発表した。SDHC対応のSD系メディアやコンパクトフラッシュ、メモリースティック系メディア、xDピクチャーカードといったデジカメやスマホ、携帯ゲームといった機器で使われている各種メディアを従来よりも短時間でPCに取り込むことが可能になる。

転送速度が5Gbps（理論値）とUSB2.0の480Mbpsと比べて爆速になったUSB3.0はPC側の対応が進んで来ていたが高速転送が生かせる周辺機器としては、外付けHDDや一部のUSBメモリーくらいしかなかった。これに多くのメディアが扱えるリーダー／ライターが加わることで手軽にUSB3.0の恩恵を受けることができるようになる。

今回発表されたのは、USB3.0ケーブルとカードリーダー本体が分かれるタイプの「BSCR09U3」シリーズ（3,240円）、USB3.0コネクタをカードリーダー本体に内蔵している「BSCRD04U3」シリーズ（2,690円）だ。共にホワイトとブラックのカラーバリエーションが用意される（発売は3月下旬以降）。

■リリースページ
■バッファローコクヨサプライ




■バッファローの記事をもっと見る
・約283gでカバンに入る！小型キーボードの驚くべき機能
・3種類のホットキーで使いやすい！AndroidとPCで使えるキーボードの魅力
・ドラえもんもビックリの新アイテム！マウスとキーボードが合体"OPAir"
・ありそうでなかった便利機能！ファイル仕分けする画期的なHDD


サンディスク SanDisk microSDHC 32GB（microSD 32GB） 超高速クラス4  変換アダプター付 世界国内シェアNo.1 バルク品
クチコミを見る


ファイルのテキストデータの構成は、以下のとおりです。  
1行目：記事のURL  
2行目：記事の作成日時  
3行目：記事のタイトル  
4行目以降：記事の本文  
カテゴリーラベルは、textディレクトリ配下のフォルダ名にある。

---

## 2. BERTのファインチューニングと性能評価
BERTでファインチューニングと性能評価を行う場合、データを前処理し、BERTに入力可能な形式に整える。　（データセット→データローダという形式へ変換）


### 2-1. データローダ作成

In [ ]:
# データローダーの作成（データセットからミニバッチを作る）
dataset_for_loader = [
    {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)},
]
loader = DataLoader(dataset_for_loader, batch_size=2)

# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ## ファインチューニングではここでミニバッチ毎の処理を行う

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [ ]:
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)
#shuffle= Trueでデータセットからランダムにデータ抜き出し、ミニバッチ作成

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}
# batch 1
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}


### 2-2. データ前処理
正解ラベルである９つのカテゴリーリストを作成した後、
データセットの学習/検証/テストデータに分割し、それぞれのデータローダを作成
前処理では、各記事の文章を符号化し、BERTに入力できるようにする。
ここでトークン数は、１２８とする。（最大512でトークン数が多ければ、パフォーマンスはあがるが、学習時間も長くなる）

---

以下のキーを辞書にして、データローダに入力できる形式に整形


*   input_ids
*   attention_mask
*   token_type_ids
*   labels

※BERTはラベルのデータを引数labelsとして受け入れるので、labelではなくlabelsとすることに注意

In [ ]:
# カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'./text/{category}/{category}*'):
        lines = open(file).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目（記事文章）からを抜き出す。
        encoding = tokenizer(
            text,
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label # ラベルを追加
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)
# tqdmは「タカドゥム」でfor文などの時間がかかる処理を可視化できるライブラリパッケージ
# glob.globで特定パターンにマッチするファイル取得

100%|██████████| 9/9 [00:37<00:00,  4.22s/it]


In [ ]:
# 前処理後のdataset中身確認
print(dataset_for_loader[0])

{'input_ids': tensor([    2,  4443,  2568,    14,  3200,    34,  1359,     6,  7800,   118,
            5,  2986,  7401,     9,   654,  2568,    14,  2383,   308,    10,
            8,  7800,    13,  8461,  3318,     6, 21069,     7,  2657,    20,
           10,  1879,     5,  2986,   478,    11,  8768,   895,     7, 21186,
            8,  1778,  3171,   312,     6,  4794, 21914,  2855,     9,  8027,
           16,  1281,    10,   120,    12,    31,     8,   373,   265,     9,
            6,  4443,  2568,    14,   130,  5589,    14,   130,     6,  3472,
           11,  9709,   312,     6,  6905,    12,  1879,     5,  4423,  3811,
          126,  9031,    16,  1497,   999, 30620, 28827,    18,   802, 28545,
        28637,     8,  8709,    16,    28,  8027,    80,   281,   103, 28507,
           13,     5,  2855,     7,   569,   335,   558,     9,  1037,    11,
         7105,     5,    12,  5198,   205,    29,  2935,    36,    76,    19,
          174,     7,  8461,    10,   306,    40, 

### 2-3.  データセットの分割
バッチサイズの工夫として、

*   学習データではBERTオリジナルの論文を参考に３２と設定
*   検証データとテストデータでは、損失の勾配を計算する必要がないため、バッチサイズを２５６と設定



In [ ]:
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=128, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

## 3. PyTorch Lightningによるファインチューニングとテスト
学習データと検証データを用いてBERTをファインチューニングし、その後にテストデータを用いてモデルの性能を評価する。  
PyTorch Lughtningを使用することでコード量を減らすことができる。

In [ ]:
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。(BERTの元論文参考)
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
# ファインチューニングの設定
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

RuntimeError: ignored

この後のベストモデルの検証データでの損失値を見るためにベストモデルとそのパズの保存を行う。

In [ ]:
# ベストモデルの保存
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

In [ ]:
# 損失値の時間変化の可視化（TensorBoard利用）
%load_ext tensorboard
%tensorboard --logdir ./

## 4. モデルの評価

In [ ]:
# ファインチューニングで得たモデルをテストデータで評価
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.4f}')

ファインチューニングしたモデルは、88.8％の精度で分類できていることがわかりました。  
(学習データ：バッチサイズ６４、学習率1e-5)

In [ ]:
# ファインチューニングしたモデルの保存と読み込み
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers') 

In [ ]:
# ディレクトリを指定し、transformerのモデルとして直接読みこませられるように設定
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)

## 5. 考察
ニュース記事を９つのカテゴリーへ分類するタスクの精度は、約88.8%となった。  
使用したモデルは、ネットに精度が出やすいと記載があり、bert-base-japanese-whole-word-maskingを選択し、  
モデリングを行う際に学習データのバッチサイズ６４と設定した。

精度を上げていくには、ハイパーパラメーターチューニングを考えているが、時間の関係上できていない。  
もし、最大系列長、ミニバッチサイズ、学習係数、ウォームアップ時間の割合、早期終了基準のパラメーターのグリッドリサーチにより、最適なハイパーパラメータを見つければ、精度は上げられるのではないかと考える。


## 参考文献

1.   [BERTの使い方 - 日本語pre-trained modelsをfine tuningして分類問題を解く](https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3)
2.   [BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)
3.   [pythonによる日本語前処理備忘録](https://datumstudio.jp/blog/python%E3%81%AB%E3%82%88%E3%82%8B%E6%97%A5%E6%9C%AC%E8%AA%9E%E5%89%8D%E5%87%A6%E7%90%86%E5%82%99%E5%BF%98%E9%8C%B2/)

